In [2]:
from mxnet import autograd, nd
from mxnet.gluon import nn
# X:输入数组，K：核数组，即过滤器
def corr2d(X, K): # 本函数已保存在d2lzh包中⽅便以后使⽤
    h, w = K.shape  #得到核数组的高和宽两个属性
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 矩阵 * 矩阵 ：对应元素相乘
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [4]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))
        
     #直接调用corr2d函数 最后加上偏差   
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

In [5]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
# 构造卷积核
K = nd.array([[1, -1]])

In [7]:
# 做核相关运算
Y = corr2d(X, K)
Y
#我们将从⽩到⿊的边缘和从⿊到⽩的边缘分别检测成了1和-1。其余部分的输出全是0
#卷积层可通过重复使⽤卷积核有效地表征局部空间


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [8]:
# 构造⼀个输出通道数为1（将在“多输⼊通道和多输出通道”⼀节介绍通道），核数组形状是(1, 2)的⼆
# 维卷积层
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

In [9]:
# ⼆维卷积层使⽤4维输⼊输出，格式为(样本, 通道, ⾼, 宽)，这⾥批量⼤⼩（批量中的样本数）和通
# 道数均为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

In [10]:
for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    # 简单起⻅，这⾥忽略了偏差
    # 计算梯度更新权重
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


In [11]:
 conv2d.weight.data().reshape((1, 2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>